Source: https://forum.image.sc/t/using-dask-arrays-with-pyopencl/56936

In [1]:
import numpy as np
import pyopencl as cl
import pyopencl.array as cla

#Make some simple data
x = np.array([[1, 2, 3, 4, 5, 6]])
img = np.vstack([np.zeros_like(x), x, x, x, np.zeros_like(x)]).astype(np.float32)
# %%
#Initialize pyopencl
ctx = cl.create_some_context()
cq = cl.CommandQueue(ctx)

#%%
#Simple GPU function
def test_gpu(img, queue=cq):
    #send img to gpu
    img_g = cla.to_device(queue, img)
    #create space on GPU for output
    out_g = cla.empty_like(img_g)
    #simple math
    out_g = img_g * img_g
    #return numpy array from GPU
    return out_g.get()
#%%
#Analogous non-GPU function
def test_cpu(img):
    out_c = img * img
    return out_c

In [2]:
import dask.array as da

#make a dask array of 5 slices of img
dask_data = []
for i in range(5):
    darray = da.from_array(img)
    stack = da.stack(darray, axis=0)
    dask_data.append(stack)

data = da.stack(dask_data)
data = data.rechunk((1, 5, 6))
data

dask.array<rechunk-merge, shape=(5, 5, 6), dtype=float32, chunksize=(1, 5, 6), chunktype=numpy.ndarray>

In [3]:
test3 = test_cpu(data[0])
type(test3)

dask.array.core.Array

In [4]:
test4 = test_gpu(data[0])
test4

AttributeError: 'Array' object has no attribute 'strides'

In [ ]:
map_with_cpu = data.map_blocks(test_cpu, dtype="float32", meta=np.array((), dtype=np.float32))

In [ ]:
map_with_cpu

In [ ]:
map_with_gpu = data.map_blocks(test_gpu, dtype="float32", meta=np.array((), dtype=np.float32))
map_with_gpu